# Working with the RGI files

The glacier outlines obtained from the [Randolph Glacier Inventory](https://www.glims.org/RGI/) are the reference dataset for global and regional applications in OGGM. The current version is V6, and OGGM ships with a slightly modified version which we called `62`.

## Download the glacier outlines 

 To download this version, simply do: 

In [ ]:
# this might take a couple of minutes!
from oggm import utils
utils.get_rgi_dir(version='62')  # path to the data after download

## Access a region file

The RGI is divided in 19 regions (and many more sub-regions, not plotted here):

![rgi-map](https://www.researchgate.net/profile/Tobias_Bolch/publication/264125572/figure/fig1/AS:295867740377088@1447551774164/First-order-regions-of-the-RGI-with-glaciers-shown-in-red-Region-numbers-are-those-of.png)
*Source: [the RGI consortium](http://www.glims.org/RGI/randolph60.html)*

In [ ]:
fr = utils.get_rgi_region_file(11, version='62')  # Central Europe

The RGI region files are [shapefiles](https://en.wikipedia.org/wiki/Shapefile), a vector format commonly used in GIS applications. The library of choice to read shapefiles in python is [geopandas](http://geopandas.org/):

In [ ]:
import geopandas as gpd
gdf = gpd.read_file(fr)

## The RGI files and their attributes 

The `gdf` variable is a `GeoDataFrame`, i.e. you can use most of the tools you know from pandas' `DataFrames`:

In [ ]:
len(gdf)

In [ ]:
gdf.head()

In [ ]:
gdf[['Area']].plot(kind='hist', bins=100, logy=True);

In [ ]:
gdf[['Aspect']].plot(kind='hist', bins=45);

## Selecting glaciers per attribute

You may want to select all glaciers in the subregion 2 (Pyrenees):

In [ ]:
gdf_sel = gdf.loc[gdf.O2Region == '2']

In [ ]:
'Glacier area in the Pyrenees: {} km2'.format(gdf_sel.Area.sum())

## Selecting glaciers in a basin 

Let's use a file shipped with OGGM for a start: 

In [ ]:
path = utils.get_demo_file('rofental_hydrosheds.shp')
basin = gpd.read_file(path)

In [ ]:
basin.plot();

And select all glaciers within this shape:

In [ ]:
import shapely.geometry as shpg
in_bas = [basin.geometry.contains(shpg.Point(x, y))[0] for
          (x, y) in zip(gdf.CenLon, gdf.CenLat)]
gdf_sel = gdf.loc[in_bas]

In [ ]:
ax = basin.plot();
gdf_sel.plot(ax=ax, edgecolor='k');

## Select glaciers by their ID 

Each glacier in the RGI has a unique ID. It is sometimes difficult to find out which one, but some tools can help you out. For example, the [GLIMS viewer](https://www.glims.org/maps/glims) allows to select glaciers and then see their ID. For example, the Aletsch Glacier in the Swiss Alps:

In [ ]:
al = utils.get_rgi_glacier_entities(['RGI60-11.01450'], version='62')
al.plot();

## Use the RGI files to start an OGGM run 

RGI files can be given as input to OGGM to make a run:

In [ ]:
from oggm import cfg, workflow, tasks
cfg.initialize(logging_level='WARNING')
cfg.PARAMS['continue_on_error'] = True
cfg.PARAMS['use_multiprocessing'] = True
cfg.PARAMS['border'] = 80
cfg.PATHS['working_dir'] = utils.gettempdir(dirname='OGGM-Rofental', reset=True)

gdirs = workflow.init_glacier_directories(gdf_sel, from_prepro_level=4)

In [ ]:
workflow.execute_entity_task(tasks.run_random_climate, gdirs, nyears=100,
                             y0=2009, halfsize=10, output_filesuffix='_2000')
ds2000 = utils.compile_run_output(gdirs, filesuffix='_2000')

In [ ]:
ds2000.sum(dim='rgi_id').volume.plot();

This shows the summed up volume evolution of all glaciers of the Rofental basin.

## What's next?

- return to the [OGGM documentation](https://docs.oggm.org)
- back to the [table of contents](welcome.ipynb)